# **Final assigment - Capstone project**

### **Introduction** 

My project is about finding a proper way to buy a house in London.
Instead of just pick a nice one, I'd like to find the best one according to my preferences.
I am going to use Foresquare and clustering to solve the problem and get the neighborhood where I might to look around to live.

Also I have preferences, like
- least populated area where 
- the average price is lower then £350.000
- it should be safe for sure, and
- it would be just nice if the poeple around were not too old

So I have strict requests to find the neighborhood where I should start the search.

To solve the problem I would create a dataset which will contain useful information about London's districts and I will use this one to get data from Foursquare.

Those who wants to buy a new house might be interested on the method I applied here.



### **Data**

#### Administraive areas

The table (requested from [https://www.doogal.co.uk/AdministrativeAreas.php](https://www.doogal.co.uk/AdministrativeAreas.php) ) conatains the postcode data filtered by administrative area that is Districts.
The dataset contain the codes of every districts and their geographical data i.e. Latitude and Longitude.

#### London postcodes

The table requested from [https://www.doogal.co.uk/london_postcodes.php](https://www.doogal.co.uk/london_postcodes.php) 

This is a complete list of London postcode districts with their Ordnance Survey coordinates and longitude and latitude.
It seems cool but notice the Latitude and Longitude data are go with the Postcode which is a bit deep level for this examination.

On the other hand it contains the data of **District Code**, **Ward Code**, **District**, **Ward**, **Rural/urban**.
Those data would be useful:
- District Code and the Ward Code for merging with other datasets,
- Rural/urban for the analysis

#### Ward level geographical data

The table requested from [http://geoportal.statistics.gov.uk/datasets/wards-december-2018-full-clipped-boundaries-gb/data](
http://geoportal.statistics.gov.uk/datasets/wards-december-2018-full-clipped-boundaries-gb/data) 

To get geographical data about wards. It does not contains ward level data so I could merge it with the *London postcodes* dataset

#### London borough profile

The table requested from [https://data.london.gov.uk/dataset/london-borough-profiles](
https://data.london.gov.uk/dataset/london-borough-profiles) 

Borough profile displays data for that borough, plus either Inner or Outer London, London and a national comparator (usually England where data is available). The data is set out across 11 themes covering most of the key indicators relating to demographic, economic, social and environmental data. 

I would mainly use **GLA Population Estimate 2017**, **GLA Household Estimate 2017**, 
**Population density (per hectare) 2017**, **Average Age, 2017** columns to help me to get closer to the desirable new House.

#### Recorded crime summary

The table requested from [https://data.london.gov.uk/dataset/recorded_crime_summary](https://data.london.gov.uk/dataset/recorded_crime_summary)

This data counts the number of crimes at three different geographic levels of London (borough, ward, LSOA) per month, according to crime type. 
I will not show all the data provided, because I will summarize them i.e. I need only one number per wards to help me compare the safety of wards

#### House Price Index - HPI

The table requested from [https://data.london.gov.uk/dataset/uk-house-price-index](https://data.london.gov.uk/dataset/uk-house-price-index)

The UK House Price Index (UK HPI) captures changes in the value of residential properties.

The UK HPI uses sales data collected on residential housing transactions, whether for cash or with a mortgage.

The **Average price** and the **Sales volume** are available from this table which I definiately need to solve the problem

---

#### Get the neighborhood

Once I collected all the needed data and created the dataset I want, I will start to examine it to find the proper neighborhood which is:
- least crowded, so least populated
- the average price is lower then £350.000
- safe
- the area is youthful

Once I get the neighborhood which is close enough to my expectation (filtered data) I will use the data to make a map and then to make clusters to see the venues.

Based on the filtered data and the venues I am going to have the idea where to start the searching.

---
### **1. Get the data**

First import the neccessary libraries

In [326]:
import pandas as pd
import numpy as np
import requests

Get [Administrative areas](https://www.doogal.co.uk/AdministrativeAreas.php)

Since we need only three columns from it, I extract *District Code*, *Latitude*, *Longitude*
Also I rename those columns to show they belong to Districts

In [327]:
df_district= pd.read_csv('D:/Programming/Python/Jupyter_notebooks/IBM course/Final_assignment/Admin areas.csv')
df_district = df_district[['District Code','Latitude','Longitude']]
df_district.rename(index=str, columns={'Latitude':'District Latitude','Longitude':'District Longitude'}, inplace=True)
df_district.head()

,District Code,District Latitude,District Longitude
0,S12000033,57.149502,-2.132760
1,S12000034,57.354000,-2.322610
2,E07000223,50.833302,-0.283848
3,E07000026,54.713402,-3.361390
4,E07000032,53.037601,-1.424500


Get [London postcodes](https://www.doogal.co.uk/london_postcodes.php) then check the dataset

In [328]:
df_london = pd.read_csv('D:/Programming/Python/Jupyter_notebooks/IBM course/Final_assignment/London postcodes.csv')

# check the dataset
df_london.head(3)

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,Census output area,Constituency Code,Index of Multiple Deprivation,Quality,User Type,Last updated,Nearest station,Distance to station,Postcode area,Postcode district
0,BR1 1AA,Yes,51.401546,0.015415,540291,168873,TQ402688,Greater London,Bromley,Bromley Town,...,E00003264,E14000604,20532,1,0,2019-02-23,Bromley South,0.218257,BR,BR1
1,BR1 1AB,Yes,51.406333,0.015208,540262,169405,TQ402694,Greater London,Bromley,Bromley Town,...,E00003255,E14000604,10169,1,0,2019-02-23,Bromley North,0.253666,BR,BR1
2,BR1 1AD,No,51.400057,0.016715,540386,168710,TQ403687,Greater London,Bromley,Bromley Town,...,E00003264,E14000604,20532,1,1,2019-02-23,Bromley South,0.044559,BR,BR1


In [329]:
df_london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319868 entries, 0 to 319867
Data columns (total 43 columns):
Postcode                          319868 non-null object
In Use?                           319868 non-null object
Latitude                          319868 non-null float64
Longitude                         319868 non-null float64
Easting                           319868 non-null int64
Northing                          319868 non-null int64
Grid Ref                          319868 non-null object
County                            319868 non-null object
District                          319868 non-null object
Ward                              319868 non-null object
District Code                     319868 non-null object
Ward Code                         319868 non-null object
Country                           319868 non-null object
County Code                       319868 non-null object
Constituency                      319868 non-null object
Introduced                        

In [330]:
df_london.shape

(319868, 43)

Create a shorter dataset by get just the relevant columns and roll them up, so I have only one row for each Ward

In [331]:
df_london = df_london.groupby(['District Code','Ward Code','District','Ward','Rural/urban']).apply(', '.join).reset_index()
df_london.drop(0,axis=1, inplace=True)

df_london.head()

,District Code,Ward Code,District,Ward,Rural/urban
0,E09000001,E05009288,City of London,Aldersgate,Urban major conurbation
1,E09000001,E05009289,City of London,Aldgate,Urban major conurbation
2,E09000001,E05009290,City of London,Bassishaw,Urban major conurbation
3,E09000001,E05009291,City of London,Billingsgate,Urban major conurbation
4,E09000001,E05009292,City of London,Bishopsgate,Urban major conurbation


Let's check the size of the resulting dataframe

In [332]:
df_london.shape

(689, 5)

Let's merge the df_london and df_district datasets. Since I need every row from df_london but from df_district only those where there is a connection between the *District Codes*, I am going to use **pd.merge()** 

In [333]:
df_full = pd.merge(df_london, df_district, how='left',on='District Code')
df_full.head()

,District Code,Ward Code,District,Ward,Rural/urban,District Latitude,District Longitude
0,E09000001,E05009288,City of London,Aldersgate,Urban major conurbation,51.514301,-0.091442
1,E09000001,E05009289,City of London,Aldgate,Urban major conurbation,51.514301,-0.091442
2,E09000001,E05009290,City of London,Bassishaw,Urban major conurbation,51.514301,-0.091442
3,E09000001,E05009291,City of London,Billingsgate,Urban major conurbation,51.514301,-0.091442
4,E09000001,E05009292,City of London,Bishopsgate,Urban major conurbation,51.514301,-0.091442


In [334]:
df_full[df_full['Ward Code']=='E05009289'].groupby('Ward').count()

,District Code,Ward Code,District,Rural/urban,District Latitude,District Longitude
Ward,,,,,,
Aldgate,1,1,1,1,1,1


In [335]:
df_full.loc[:,'Ward Code'].head(1)

0    E05009288
Name: Ward Code, dtype: object

Get [Ward level geographical data](http://geoportal.statistics.gov.uk/datasets/wards-december-2018-full-clipped-boundaries-gb/data)

Since there are a lot of columns I do not need right now, I am going to keep only *wd18cd*, *lat*, *long* columns and rename them as *Ward Code*, *Ward Latitude* and *Ward Longitude*. *Ward Code* will be used as the key to merge with df_full dataset

In [336]:
df_ward = pd.read_csv('D:/Programming/Python/Jupyter_notebooks/IBM course/Final_assignment/Wards_December_2018_Full_Clipped_Boundaries_GB.csv')
df_ward.head(3)

,objectid,wd18cd,wd18nm,wd18nmw,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
0,1,E05000026,Abbey,,544434,184378,0.081291,51.539822,1.256366e+06,8051.446748
1,2,E05000027,Alibon,,549247,185196,0.150987,51.545921,1.364442e+06,6353.917640
2,3,E05000028,Becontree,,546863,185869,0.116912,51.552601,1.288082e+06,6341.645817


In [337]:
df_ward[df_ward['wd18cd']=='E05000109'].groupby('wd18cd').count()

,objectid,wd18nm,wd18nmw,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
wd18cd,,,,,,,,,
E05000109,1,1,1,1,1,1,1,1,1


In [338]:
df_ward = df_ward[['wd18cd','lat','long']]
df_ward.head(3)

,wd18cd,lat,long
0,E05000026,51.539822,0.081291
1,E05000027,51.545921,0.150987
2,E05000028,51.552601,0.116912


In [339]:
df_ward.shape

(8652, 3)

In [340]:
df_ward.rename(columns={'wd18cd':'Ward Code','lat':'Ward Latitude','long':'Ward Longitude'}, inplace=True)
df_ward.head(3)

,Ward Code,Ward Latitude,Ward Longitude
0,E05000026,51.539822,0.081291
1,E05000027,51.545921,0.150987
2,E05000028,51.552601,0.116912


In [341]:
df_full = pd.merge(df_full, df_ward, how='left', on='Ward Code')

#check the dataset
df_full.head(3)

,District Code,Ward Code,District,Ward,Rural/urban,District Latitude,District Longitude,Ward Latitude,Ward Longitude
0,E09000001,E05009288,City of London,Aldersgate,Urban major conurbation,51.514301,-0.091442,51.518951,-0.09645
1,E09000001,E05009289,City of London,Aldgate,Urban major conurbation,51.514301,-0.091442,51.513760,-0.07896
2,E09000001,E05009290,City of London,Bassishaw,Urban major conurbation,51.514301,-0.091442,51.516861,-0.09266


Get [London borough profile](https://data.london.gov.uk/dataset/london-borough-profiles)

The data is on the 'Data' sheet in an excel file. Also, after an examination I realized I do not need all of the columns but just a few so I grab only those. The first row contains nothing so just skeep it.
I need to rename the *New code* column too, which is our *District Code*

In [342]:
df_profile = pd.read_excel("D:/Programming/Python/Jupyter_notebooks/IBM course/Final_assignment/london-borough-profiles.xlsx", 
              sheet_name='Data',index_col=None, usecols= ['New code','GLA Population Estimate 2017',
                                                       'GLA Household Estimate 2017','Population density (per hectare) 2017',
                                                       'Average Age, 2017'], skiprows=[1])
df_profile.rename(index=str, columns={'New code':'District Code'}, inplace=True)
df_profile.head(2)

,District Code,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017"
0,E09000001,8800,5325.82,30.3037,43.2
1,E09000002,209000,78188.4,57.8822,32.9


Let's merge the datasets and check it

In [343]:
df_full = pd.merge(df_full, df_profile, how='left',on='District Code')
df_full.head(3)

,District Code,Ward Code,District,Ward,Rural/urban,District Latitude,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017"
0,E09000001,E05009288,City of London,Aldersgate,Urban major conurbation,51.514301,-0.091442,51.518951,-0.09645,8800,5325.82,30.3037,43.2
1,E09000001,E05009289,City of London,Aldgate,Urban major conurbation,51.514301,-0.091442,51.513760,-0.07896,8800,5325.82,30.3037,43.2
2,E09000001,E05009290,City of London,Bassishaw,Urban major conurbation,51.514301,-0.091442,51.516861,-0.09266,8800,5325.82,30.3037,43.2


Get [Recorded crime summary](https://data.london.gov.uk/dataset/recorded_crime_summary)

I will use only the data of year 2017 (it starts with 201702) to be consistent with the other data.

We have *WardCode* in there but after I tried it I realised it is not as good to use to merge as the column *Borough* so I keep that one.

Also I do not need to know this time what kind of crime has been commited so i just create a **Total** column for them.

In [344]:
df_crime = pd.read_csv("D:/Programming/Python/Jupyter_notebooks/IBM course/Final_assignment/MPS Ward Level Crime (most recent 24 months).csv")
df_crime.head(2)

,WardCode,WardName,Borough,Major Category,Minor Category,201702,201703,201704,201705,201706,...,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901
0,E05000026,Abbey,Barking and Dagenham,Burglary,Burglary - Business and Community,4,6,5,1,1,...,6,3,4,8,1,5,2,4,7,2
1,E05000026,Abbey,Barking and Dagenham,Burglary,Burglary - Residential,1,7,12,12,3,...,5,5,4,6,3,6,6,4,8,5


In [345]:
df_crime.columns.values

array(['WardCode', 'WardName', 'Borough', 'Major Category',
       'Minor Category', '201702', '201703', '201704', '201705', '201706',
       '201707', '201708', '201709', '201710', '201711', '201712',
       '201801', '201802', '201803', '201804', '201805', '201806',
       '201807', '201808', '201809', '201810', '201811', '201812',
       '201901'], dtype=object)

In [346]:
df_crime = df_crime[['Borough','201702','201703',
               '201704','201705','201706','201707','201708','201709','201710','201711','201712']]
df_crime.head()

,Borough,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712
0,Barking and Dagenham,4,6,5,1,1,6,5,0,2,4,4
1,Barking and Dagenham,1,7,12,12,3,5,4,3,3,4,3
2,Barking and Dagenham,0,0,1,2,3,0,1,0,2,0,0
3,Barking and Dagenham,5,4,6,6,7,3,6,4,9,3,4
4,Barking and Dagenham,3,2,3,0,5,0,2,3,3,1,2


To create the **Total** column I am using *.loc[]* and aggregate all the columns containing crime data.

Also I need each borough only once so I aggregate the rows as well.


In [347]:
df_crime.loc[:,'Total'] = df_crime[['201702','201703','201704','201705','201706','201707',
                                    '201708','201709','201710','201711','201712']].sum(axis=1)
df_crime = df_crime[['Borough','Total']].groupby('Borough', as_index=False).agg(
                            {'Total':'sum'})
df_crime.sort_values('Borough',ascending=True)
df_crime.head(3)

,Borough,Total
0,Barking and Dagenham,17019
1,Barnet,24213
2,Bexley,13480


In [348]:
df_crime.shape

(32, 2)

Rename the columns

In [349]:
df_crime.rename(index=str, columns={'Borough':'District','Total':'Crime volume in 2017'}, inplace=True)
df_crime.head(3)

,District,Crime volume in 2017
0,Barking and Dagenham,17019
1,Barnet,24213
2,Bexley,13480


Sort and check the result

In [350]:
df_crime.sort_values('District')

,District,Crime volume in 2017
0,Barking and Dagenham,17019
1,Barnet,24213
2,Bexley,13480
3,Brent,27099
4,Bromley,20892
5,Camden,34964
6,Croydon,27431
7,Ealing,25982
8,Enfield,23014
9,Greenwich,23147


In [351]:
df_full.sort_values('District')['District'].unique()

array(['Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley',
       'Camden', 'City of London', 'Croydon', 'Ealing', 'Enfield',
       'Greenwich', 'Hackney', 'Hammersmith and Fulham', 'Haringey',
       'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Islington',
       'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth',
       'Lewisham', 'Merton', 'Newham', 'Redbridge',
       'Richmond upon Thames', 'Southwark', 'Sutton', 'Tower Hamlets',
       'Waltham Forest', 'Wandsworth', 'Westminster'], dtype=object)

Be sure, every district has a value

In [352]:
df_crime[df_crime['Crime volume in 2017'].isnull()]

,District,Crime volume in 2017


Merge the datasets

In [353]:
df_full = pd.merge(df_full, df_crime, how='left',on='District')
df_full.head(3)

,District Code,Ward Code,District,Ward,Rural/urban,District Latitude,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017
0,E09000001,E05009288,City of London,Aldersgate,Urban major conurbation,51.514301,-0.091442,51.518951,-0.09645,8800,5325.82,30.3037,43.2,NaN
1,E09000001,E05009289,City of London,Aldgate,Urban major conurbation,51.514301,-0.091442,51.513760,-0.07896,8800,5325.82,30.3037,43.2,NaN
2,E09000001,E05009290,City of London,Bassishaw,Urban major conurbation,51.514301,-0.091442,51.516861,-0.09266,8800,5325.82,30.3037,43.2,NaN


I have just found out, that the crime dataset did not contain any row on *City of London* so I got **NaN** for these rows.

I do not want to get rid of these rows so I decided to replace those **NaN**-s with the mean

In [354]:
df_full[df_full['Crime volume in 2017'].isnull()].groupby('District').count()

,District Code,Ward Code,Ward,Rural/urban,District Latitude,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017
District,,,,,,,,,,,,,
City of London,25,25,25,25,25,25,25,25,25,25,25,25,0


In [355]:
df_full['Crime volume in 2017']=df_full['Crime volume in 2017'].fillna(value=df_full['Crime volume in 2017'].mean())
df_full.head(3)

,District Code,Ward Code,District,Ward,Rural/urban,District Latitude,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017
0,E09000001,E05009288,City of London,Aldersgate,Urban major conurbation,51.514301,-0.091442,51.518951,-0.09645,8800,5325.82,30.3037,43.2,23579.766566
1,E09000001,E05009289,City of London,Aldgate,Urban major conurbation,51.514301,-0.091442,51.513760,-0.07896,8800,5325.82,30.3037,43.2,23579.766566
2,E09000001,E05009290,City of London,Bassishaw,Urban major conurbation,51.514301,-0.091442,51.516861,-0.09266,8800,5325.82,30.3037,43.2,23579.766566


In [356]:
df_full[df_full['Crime volume in 2017'].isnull()].groupby('District').count()

,District Code,Ward Code,Ward,Rural/urban,District Latitude,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017
District,,,,,,,,,,,,,


Let's check it in that way too

In [357]:
df_full[['District','Crime volume in 2017']].groupby('District').mean()

,Crime volume in 2017
District,
Barking and Dagenham,17019.000000
Barnet,24213.000000
Bexley,13480.000000
Brent,27099.000000
Bromley,20892.000000
Camden,34964.000000
City of London,23579.766566
Croydon,27431.000000
Ealing,25982.000000


Get [House Price Index](https://data.london.gov.uk/dataset/uk-house-price-index)

I will get the data in two steps:
- in the first step I will get the *average price* which is on the sheet Average price
- in the second step I get the *sales volume* from the Sales volume sheet

From those excel sheets I need only the **Districts**, their **Districts Code** and the close data of 2017 which is **2017-12-01**

Then transpose the dataset and rename the columns

In [358]:
df_avghp = pd.read_excel("D:/Programming/Python/Jupyter_notebooks/IBM course/Final_assignment/UK House price index-v2.xls",
                        sheet_name='Average price',usecols='A:AH')
df_avghp.rename(index=str, columns={'Unnamed: 0':'Date'}, inplace=True)
df_avghp.head()

,Date,City of London,Barking & Dagenham,Barnet,Bexley,Brent,Bromley,Camden,Croydon,Ealing,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,NaT,E09000001,E09000002,E09000003,E09000004,E09000005,E09000006,E09000007,E09000008,E09000009,...,E09000024,E09000025,E09000026,E09000027,E09000028,E09000029,E09000030,E09000031,E09000032,E09000033
1,1995-01-01,91449,50460.2,93284.5,64958.1,71306.6,81671.5,120933,69158.2,79885.9,...,82070.6,53539.3,72189.6,109326,67885.2,71537,59865.2,61319.4,88559,133025
2,1995-02-01,82202.8,51085.8,93190.2,64787.9,72022.3,81657.6,119509,68951.1,80897.1,...,79982.7,53153.9,72141.6,111103,64799.1,70893.2,62318.5,60252.1,88641,131468
3,1995-03-01,79120.7,51269,92247.5,64367.5,72015.8,81449.3,120282,68712.4,81379.9,...,80661.7,53458.3,72501.4,107325,65763.3,70306.8,63938.7,60871.1,87124.8,132260
4,1995-04-01,77101.2,53133.5,90762.9,64277.7,72965.6,81124.4,120098,68610,82188.9,...,79990.5,54479.8,72228.6,106875,63073.6,69411.9,66233.2,60971.4,87026,133370


In [359]:
df_avghp = df_avghp.iloc[[0,pd.to_numeric(df_avghp[df_avghp.iloc[:,0]=='2017-12-01'].index[0],errors='coerce')],:]

df_avghp

,Date,City of London,Barking & Dagenham,Barnet,Bexley,Brent,Bromley,Camden,Croydon,Ealing,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,NaT,E09000001,E09000002,E09000003,E09000004,E09000005,E09000006,E09000007,E09000008,E09000009,...,E09000024,E09000025,E09000026,E09000027,E09000028,E09000029,E09000030,E09000031,E09000032,E09000033
276,2017-12-01,778058,292911,538469,340803,494534,445042,844372,373982,469142,...,516974,349361,416682,649487,484735,377968,465875,442017,596715,1.09717e+06


In [360]:
df_avghp = df_avghp.transpose().reset_index()
df_avghp.drop([0], inplace=True)
df_avghp.drop('index',axis=1, inplace=True)
df_avghp.rename(index=str, columns={'0':'District Code', '276':'AVG price on 2017-12-01'}, inplace=True)
df_avghp.head()

,District Code,AVG price on 2017-12-01
1,E09000001,778058
2,E09000002,292911
3,E09000003,538469
4,E09000004,340803
5,E09000005,494534


Merge the datasets

In [361]:
df_full = pd.merge(df_full, df_avghp, how='left',on='District Code')
df_full.head()

,District Code,Ward Code,District,Ward,Rural/urban,District Latitude,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01
0,E09000001,E05009288,City of London,Aldersgate,Urban major conurbation,51.514301,-0.091442,51.518951,-0.09645,8800,5325.82,30.3037,43.2,23579.766566,778058
1,E09000001,E05009289,City of London,Aldgate,Urban major conurbation,51.514301,-0.091442,51.513760,-0.07896,8800,5325.82,30.3037,43.2,23579.766566,778058
2,E09000001,E05009290,City of London,Bassishaw,Urban major conurbation,51.514301,-0.091442,51.516861,-0.09266,8800,5325.82,30.3037,43.2,23579.766566,778058
3,E09000001,E05009291,City of London,Billingsgate,Urban major conurbation,51.514301,-0.091442,51.510010,-0.08265,8800,5325.82,30.3037,43.2,23579.766566,778058
4,E09000001,E05009292,City of London,Bishopsgate,Urban major conurbation,51.514301,-0.091442,51.518181,-0.08150,8800,5325.82,30.3037,43.2,23579.766566,778058


Repeat exactly the same process on the sheet Sales volume to get those data as well

In [362]:
df_volhp = pd.read_excel("D:/Programming/Python/Jupyter_notebooks/IBM course/Final_assignment/UK House price index-v2.xls",
                        sheet_name='Sales volume',usecols='A:AH')
df_volhp.rename(index=str, columns={'Unnamed: 0':'Date'}, inplace=True)
df_volhp.head()

,Date,City of London,Barking & Dagenham,Barnet,Bexley,Brent,Bromley,Camden,Croydon,Ealing,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,NaT,E09000001,E09000002,E09000003,E09000004,E09000005,E09000006,E09000007,E09000008,E09000009,...,E09000024,E09000025,E09000026,E09000027,E09000028,E09000029,E09000030,E09000031,E09000032,E09000033
1,1995-01-01,17,96,332,269,233,323,198,375,303,...,219,168,268,208,182,218,147,249,403,339
2,1995-02-01,7,95,327,207,220,326,194,342,242,...,204,189,277,212,169,177,149,226,379,251
3,1995-03-01,14,144,384,318,320,449,207,447,377,...,275,294,385,307,270,319,166,276,513,365
4,1995-04-01,7,109,304,253,249,362,174,377,270,...,230,217,317,249,169,268,204,253,451,328


In [363]:
df_volhp = df_volhp.iloc[[0,pd.to_numeric(df_volhp[df_volhp.iloc[:,0]=='2017-12-01'].index[0],errors='coerce')],:]
df_volhp = df_volhp.transpose().reset_index()
df_volhp.drop([0], inplace=True)
df_volhp.drop('index',axis=1, inplace=True)
df_volhp.rename(index=str, columns={'0':'District Code', '276':'Sales volume on 2017-12-01'}, inplace=True)
df_volhp.head()

,District Code,Sales volume on 2017-12-01
1,E09000001,20
2,E09000002,180
3,E09000003,310
4,E09000004,270
5,E09000005,161


Merge the datasets and check the result

In [364]:
df_full = pd.merge(df_full, df_volhp, how='left',on='District Code')
df_full.head()

,District Code,Ward Code,District,Ward,Rural/urban,District Latitude,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
0,E09000001,E05009288,City of London,Aldersgate,Urban major conurbation,51.514301,-0.091442,51.518951,-0.09645,8800,5325.82,30.3037,43.2,23579.766566,778058,20
1,E09000001,E05009289,City of London,Aldgate,Urban major conurbation,51.514301,-0.091442,51.513760,-0.07896,8800,5325.82,30.3037,43.2,23579.766566,778058,20
2,E09000001,E05009290,City of London,Bassishaw,Urban major conurbation,51.514301,-0.091442,51.516861,-0.09266,8800,5325.82,30.3037,43.2,23579.766566,778058,20
3,E09000001,E05009291,City of London,Billingsgate,Urban major conurbation,51.514301,-0.091442,51.510010,-0.08265,8800,5325.82,30.3037,43.2,23579.766566,778058,20
4,E09000001,E05009292,City of London,Bishopsgate,Urban major conurbation,51.514301,-0.091442,51.518181,-0.08150,8800,5325.82,30.3037,43.2,23579.766566,778058,20


In [365]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 689 entries, 0 to 688
Data columns (total 16 columns):
District Code                            689 non-null object
Ward Code                                689 non-null object
District                                 689 non-null object
Ward                                     689 non-null object
Rural/urban                              689 non-null object
District Latitude                        689 non-null float64
District Longitude                       689 non-null float64
Ward Latitude                            689 non-null float64
Ward Longitude                           689 non-null float64
GLA Population Estimate 2017             689 non-null int64
GLA Household Estimate 2017              689 non-null object
Population density (per hectare) 2017    689 non-null object
Average Age, 2017                        689 non-null float64
Crime volume in 2017                     689 non-null float64
AVG price on 2017-12-01             

Looks like there are a couple of rows which should be **int** instead of **object**.

Let's convert them

In [366]:
df_full['Sales volume on 2017-12-01'] = df_full['Sales volume on 2017-12-01'].astype('int64')
df_full['AVG price on 2017-12-01'] = df_full['AVG price on 2017-12-01'].astype('int64')
df_full['Population density (per hectare) 2017'] = df_full['Population density (per hectare) 2017'].astype('int64')
df_full['GLA Household Estimate 2017'] = df_full['GLA Household Estimate 2017'].astype('int64')

---
### **2. Explore**

Okay, that was all the data I needed. I put together so let's check them out.

Let's start with the location of the **Wards**

In [367]:
df_full['Rural/urban'].value_counts()

Urban major conurbation                655
Urban city and town                     12
Rural village                            8
Rural hamlet and isolated dwellings      8
Rural town and fringe                    6
Name: Rural/urban, dtype: int64


It was not too helpful so let's start examine the indicator columns

In [368]:
df_mean = df_full[['District','Rural/urban','GLA Population Estimate 2017','Population density (per hectare) 2017','Average Age, 2017',
         'Crime volume in 2017','AVG price on 2017-12-01','Sales volume on 2017-12-01']].groupby(['District','Rural/urban']).mean()

df_mean.sort_values('AVG price on 2017-12-01',ascending=False)

,,GLA Population Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
District,Rural/urban,,,,,,
Kensington and Chelsea,Urban major conurbation,159000,131,39.3,19468.000000,1364393,138
Westminster,Urban major conurbation,242100,112,37.7,52144.000000,1097172,215
Camden,Urban major conurbation,242500,111,36.4,34964.000000,844371,173
City of London,Urban major conurbation,8800,30,43.2,23579.766566,778057,20
Hammersmith and Fulham,Urban major conurbation,185300,113,35.7,19821.000000,745314,200
Richmond upon Thames,Urban major conurbation,197300,34,38.8,12269.000000,649486,222
Islington,Urban major conurbation,231200,155,34.8,29694.000000,639409,135
Wandsworth,Urban major conurbation,321000,93,35.0,23065.000000,596714,338
Hackney,Urban major conurbation,274300,143,33.1,29288.000000,557031,269


That's too long to make a proper insight so let's put it another way without the **Districts**

In [369]:
df_mean=df_full[['Rural/urban','GLA Population Estimate 2017','Population density (per hectare) 2017','Average Age, 2017',
         'Crime volume in 2017','AVG price on 2017-12-01','Sales volume on 2017-12-01']].groupby(['Rural/urban']).mean()

df_mean.sort_values('AVG price on 2017-12-01',ascending=False)

,GLA Population Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
Rural/urban,,,,,,
Urban major conurbation,270878.167939,72.919084,36.414504,23762.046052,532301.68855,240.861069
Urban city and town,304841.666667,31.666667,38.400000,20333.833333,441716.25000,307.916667
Rural town and fringe,289950.000000,42.500000,38.416667,21732.166667,431428.00000,321.166667
Rural village,288100.000000,33.500000,38.137500,19157.000000,426897.62500,267.875000
Rural hamlet and isolated dwellings,287125.000000,26.875000,39.125000,19333.000000,391680.37500,303.500000


Observe the tbale we got:
..* GLA Population Estimate 2017
- as you can guess, Urban city is the most poplutaed one however surprise surprise, not the rural site is the least populated but, the conurbation
..* Population density (per hectare) 2017
- That one is interesting: I expected the city to have the highest density on population but I was wrong. However the hamlets are the least crowded area
..* Average Age, 2017
- Conurbation again. Those areas have the youth
..* Crime volume in 2017 
- The most of the crimes are commited in the conurbation area. The presence of the police might have more patrol in the inner city or maybe the conurbation area is just too big to supervise
..* AVG price on 2017-12-01 
- I assume that a lot of people move to conurbation recently. This can be the explanation of the high average price, also it can explain the density
..*Sales volume on 2017-12-01
- Despite the comments above the conurbation has the least sales volume

There are a lot of questions above which are out of our topic now, because the history is not importnat right now, however it would worth a proper examination.

Now, let's focus on why we are here. 

Let's apply our requests on the dataset

In [370]:
df_target = df_full[
    (df_full['GLA Population Estimate 2017']<df_full['GLA Population Estimate 2017'].mean()) &
    (df_full['GLA Household Estimate 2017']<df_full['GLA Household Estimate 2017'].mean()) &
    (df_full['Population density (per hectare) 2017'] < df_full['Population density (per hectare) 2017'].mean()) &
    (df_full['Average Age, 2017'] < df_full['Average Age, 2017'].mean()) &
    (df_full['Crime volume in 2017'] < df_full['Crime volume in 2017'].mean()) & 
    (df_full['AVG price on 2017-12-01']<350000)]
                    
                   
df_target.head()



,District Code,Ward Code,District,Ward,Rural/urban,District Latitude,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
25,E09000002,E05000026,Barking and Dagenham,Abbey,Urban major conurbation,51.546501,0.125055,51.539822,0.081291,209000,78188,57,32.9,17019.0,292910,180
26,E09000002,E05000027,Barking and Dagenham,Alibon,Urban major conurbation,51.546501,0.125055,51.545921,0.150987,209000,78188,57,32.9,17019.0,292910,180
27,E09000002,E05000028,Barking and Dagenham,Becontree,Urban major conurbation,51.546501,0.125055,51.552601,0.116912,209000,78188,57,32.9,17019.0,292910,180
28,E09000002,E05000029,Barking and Dagenham,Chadwell Heath,Urban major conurbation,51.546501,0.125055,51.583420,0.138596,209000,78188,57,32.9,17019.0,292910,180
29,E09000002,E05000030,Barking and Dagenham,Eastbrook,Urban major conurbation,51.546501,0.125055,51.555191,0.173453,209000,78188,57,32.9,17019.0,292910,180


Check which **District** do we have after filtering the dataset

In [371]:
df_mean = df_target[['District','Rural/urban','GLA Population Estimate 2017','Population density (per hectare) 2017','Average Age, 2017',
         'Crime volume in 2017','AVG price on 2017-12-01','Sales volume on 2017-12-01']].groupby(['District','Rural/urban']).mean()

df_mean.sort_values('AVG price on 2017-12-01',ascending=False)

,,GLA Population Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
District,Rural/urban,,,,,,
Barking and Dagenham,Urban major conurbation,209000,57,32.9,17019.0,292910,180


For me this is a nice catch.

The crime volume is relatively low, it seems not overpopulated, the prices are not in the sky and the area seems youthful.

I can go with it

In [372]:
df_target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 25 to 41
Data columns (total 16 columns):
District Code                            17 non-null object
Ward Code                                17 non-null object
District                                 17 non-null object
Ward                                     17 non-null object
Rural/urban                              17 non-null object
District Latitude                        17 non-null float64
District Longitude                       17 non-null float64
Ward Latitude                            17 non-null float64
Ward Longitude                           17 non-null float64
GLA Population Estimate 2017             17 non-null int64
GLA Household Estimate 2017              17 non-null int64
Population density (per hectare) 2017    17 non-null int64
Average Age, 2017                        17 non-null float64
Crime volume in 2017                     17 non-null float64
AVG price on 2017-12-01                  17 non-null 

Let's create a dataset to mapping.



In [373]:
df_tomap=df_target[['District','Ward','Ward Latitude','Ward Longitude']].reset_index()
df_tomap.drop('index',axis=1, inplace=True)
df_tomap.head(3)

,District,Ward,Ward Latitude,Ward Longitude
0,Barking and Dagenham,Abbey,51.539822,0.081291
1,Barking and Dagenham,Alibon,51.545921,0.150987
2,Barking and Dagenham,Becontree,51.552601,0.116912


---

#### Use geopy library to get the latitude and longitude values of London.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>UK_explorer</em>, as shown below.

In [374]:
from geopy.geocoders import Nominatim
import folium

address = 'London, UK'

geolocator = Nominatim(user_agent="UK_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


#### Create a map of London with neighborhoods superimposed on top.

In [375]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, district, ward in zip(df_tomap['Ward Latitude'], df_tomap['Ward Longitude'], df_tomap['District'], df_tomap['Ward']):
    label = '{}, {}'.format(district, ward)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

----

#### Define Foursquare Credentials and Version

In [376]:
CLIENT_ID = '1HN2DYV3VM2MLMKC2DRYIW14O22AMS0UMIL4EZJY0MTQZOJB' # your Foursquare ID
CLIENT_SECRET = 'WSDV1HNYYU3NUMX1FNDGN3I0LA5JPP541NZQPPV5IW4MC03L' # your Foursquare Secret
VERSION = '20190223' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1HN2DYV3VM2MLMKC2DRYIW14O22AMS0UMIL4EZJY0MTQZOJB
CLIENT_SECRET:WSDV1HNYYU3NUMX1FNDGN3I0LA5JPP541NZQPPV5IW4MC03L


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [377]:
df_tomap.loc[0, 'Ward']

'Abbey'

Get the neighborhood's latitude and longitude values.

In [378]:
neighborhood_latitude = df_tomap.loc[0, 'Ward Latitude'] # neighborhood latitude value
neighborhood_longitude = df_tomap.loc[0, 'Ward Longitude'] # neighborhood longitude value

neighborhood_name = df_tomap.loc[0, 'Ward'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Abbey are 51.539821999999994, 0.081291.



#### Now, let's get the top 100 venues that are in Abbey within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [379]:
# type your answer here
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    VERSION, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=1HN2DYV3VM2MLMKC2DRYIW14O22AMS0UMIL4EZJY0MTQZOJB&client_secret=WSDV1HNYYU3NUMX1FNDGN3I0LA5JPP541NZQPPV5IW4MC03L&ll=51.539821999999994,0.081291&v=20190223&radius=500&limit=100'

Send the GET request and examine the resutls

In [380]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7876e5f594df76b402d2fe'},
 'response': {'headerLocation': 'Barking',
  'headerFullLocation': 'Barking, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 51.544322004499996,
    'lng': 0.08851256772088649},
   'sw': {'lat': 51.53532199549999, 'lng': 0.07406943227911351}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d1235522e5837045532e2d1',
       'name': "Nando's",
       'location': {'address': 'Unit 1, The Odeon, Longbridge Rd.',
        'lat': 51.53965459130261,
        'lng': 0.08182801881372569,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.53965459130261,
          'lng': 0.08182801881372569}],
        'distance': 41,
        'postalCode

In [381]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [382]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Nando's,Portuguese Restaurant,51.539655,0.081828
1,Cristina's,Steakhouse,51.536523,0.076672
2,The Gym London Barking,Gym,51.536193,0.078601
3,Subway,Sandwich Place,51.538688,0.080788
4,Costa coffee,Coffee Shop,51.539272,0.081341


And how many venues were returned by Foursquare?

In [383]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


---

## 2. Explore Neighborhoods in London

#### Let's create a function to repeat the same process to all the neighborhoods in London

In [384]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *london_venues*.

In [385]:
london_venues = getNearbyVenues(names=df_tomap['Ward'],
                                   latitudes=df_tomap['Ward Latitude'],
                                   longitudes=df_tomap['Ward Longitude']
                                  )



Abbey
Alibon
Becontree
Chadwell Heath
Eastbrook
Eastbury
Gascoigne
Goresbrook
Heath
Longbridge
Mayesbrook
Parsloes
River
Thames
Valence
Village
Whalebone


#### Let's check the size of the resulting dataframe

In [386]:
print(london_venues.shape)
london_venues.head()

(94, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbey,51.539822,0.081291,Nando's,51.539655,0.081828,Portuguese Restaurant
1,Abbey,51.539822,0.081291,Cristina's,51.536523,0.076672,Steakhouse
2,Abbey,51.539822,0.081291,The Gym London Barking,51.536193,0.078601,Gym
3,Abbey,51.539822,0.081291,Subway,51.538688,0.080788,Sandwich Place
4,Abbey,51.539822,0.081291,Costa coffee,51.539272,0.081341,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [387]:
london_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbey,17,17,17,17,17,17
Alibon,5,5,5,5,5,5
Becontree,5,5,5,5,5,5
Chadwell Heath,5,5,5,5,5,5
Eastbrook,5,5,5,5,5,5
Eastbury,5,5,5,5,5,5
Gascoigne,5,5,5,5,5,5
Goresbrook,6,6,6,6,6,6
Heath,8,8,8,8,8,8


#### Let's find out how many unique categories can be curated from all the returned venues

In [388]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 50 uniques categories.


## 3. Analyze Each Neighborhood

In [389]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Neighborhood,Auto Workshop,BBQ Joint,Bakery,Bar,Bus Station,Bus Stop,Business Service,Café,Chinese Restaurant,...,Pool,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Soccer Field,Sporting Goods Shop,Steakhouse,Supermarket,Train Station
0,Abbey,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,Abbey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Abbey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abbey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Abbey,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [390]:
london_onehot.shape

(94, 51)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [391]:
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped

,Neighborhood,Auto Workshop,BBQ Joint,Bakery,Bar,Bus Station,Bus Stop,Business Service,Café,Chinese Restaurant,...,Pool,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Soccer Field,Sporting Goods Shop,Steakhouse,Supermarket,Train Station
0,Abbey,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.058824,...,0.000,0.058824,0.058824,0.0,0.058824,0.000000,0.0,0.058824,0.058824,0.0
1,Alibon,0.0,0.0,0.000000,0.000000,0.000,0.200000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.200000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,Becontree,0.0,0.0,0.000000,0.000000,0.000,0.200000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.200000,0.0
3,Chadwell Heath,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000000,0.2,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,Eastbrook,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.2,0.000000,0.000000,0.0
5,Eastbury,0.2,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.200000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
6,Gascoigne,0.0,0.2,0.000000,0.000000,0.200,0.000000,0.000000,0.000000,0.200000,...,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.200000,0.0
7,Goresbrook,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000000,0.0,0.000000,0.333333,0.0,0.000000,0.000000,0.0
8,Heath,0.0,0.0,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,...,0.125,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.125000,0.0
9,Longbridge,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.200000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [392]:
london_grouped.shape

(17, 51)

#### Let's print each neighborhood along with the top 5 most common venues

In [393]:
num_top_venues = 5

for hood in london_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbey----
                  venue  freq
0              Platform  0.12
1  Fast Food Restaurant  0.12
2         Grocery Store  0.12
3    Chinese Restaurant  0.06
4              Pharmacy  0.06


----Alibon----
               venue  freq
0        Pizza Place   0.2
1           Bus Stop   0.2
2      Grocery Store   0.2
3                Pub   0.2
4  Convenience Store   0.2


----Becontree----
               venue  freq
0        Dry Cleaner   0.2
1        Supermarket   0.2
2           Bus Stop   0.2
3     Cosmetics Shop   0.2
4  Fish & Chips Shop   0.2


----Chadwell Heath----
               venue  freq
0              Hotel   0.2
1          Mini Golf   0.2
2         Restaurant   0.2
3      Grocery Store   0.2
4  Indian Restaurant   0.2


----Eastbrook----
                        venue  freq
0                        Lake   0.4
1         Sporting Goods Shop   0.2
2  Construction & Landscaping   0.2
3                        Park   0.2
4                 Pizza Place   0.0


----Eastbury----
   

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.


In [394]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [395]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey,Grocery Store,Platform,Fast Food Restaurant,Supermarket,Coffee Shop,Chinese Restaurant,Gym,Pharmacy,Pizza Place,Hotel
1,Alibon,Grocery Store,Bus Stop,Pub,Pizza Place,Convenience Store,Train Station,Cosmetics Shop,Gym / Fitness Center,Gym,Golf Course
2,Becontree,Dry Cleaner,Bus Stop,Fish & Chips Shop,Supermarket,Cosmetics Shop,Train Station,History Museum,Gym / Fitness Center,Gym,Grocery Store
3,Chadwell Heath,Hotel,Indian Restaurant,Grocery Store,Restaurant,Mini Golf,Cosmetics Shop,Gym,Golf Course,Gas Station,Fish & Chips Shop
4,Eastbrook,Lake,Park,Sporting Goods Shop,Construction & Landscaping,Train Station,Gym,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop


---

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [396]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 3, 1, 1, 2, 2, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [397]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = df_tomap
london_merged.rename(columns={'Ward':'Neighborhood'},inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

london_merged.head() # check the last columns!

,District,Neighborhood,Ward Latitude,Ward Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Abbey,51.539822,0.081291,2,Grocery Store,Platform,Fast Food Restaurant,Supermarket,Coffee Shop,Chinese Restaurant,Gym,Pharmacy,Pizza Place,Hotel
1,Barking and Dagenham,Alibon,51.545921,0.150987,2,Grocery Store,Bus Stop,Pub,Pizza Place,Convenience Store,Train Station,Cosmetics Shop,Gym / Fitness Center,Gym,Golf Course
2,Barking and Dagenham,Becontree,51.552601,0.116912,2,Dry Cleaner,Bus Stop,Fish & Chips Shop,Supermarket,Cosmetics Shop,Train Station,History Museum,Gym / Fitness Center,Gym,Grocery Store
3,Barking and Dagenham,Chadwell Heath,51.583420,0.138596,2,Hotel,Indian Restaurant,Grocery Store,Restaurant,Mini Golf,Cosmetics Shop,Gym,Golf Course,Gas Station,Fish & Chips Shop
4,Barking and Dagenham,Eastbrook,51.555191,0.173453,3,Lake,Park,Sporting Goods Shop,Construction & Landscaping,Train Station,Gym,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop


Finally, let's visualize the resulting clusters

In [400]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Ward Latitude'], london_merged['Ward Longitude'], 
                                  london_merged['Neighborhood'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

-----

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [401]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Village,Bakery,Bar,Bus Stop,Train Station,Dry Cleaner,History Museum,Gym / Fitness Center,Gym,Grocery Store,Golf Course


#### Cluster 2

In [402]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Eastbury,Auto Workshop,Pub,Grocery Store,Metro Station,History Museum,Bus Station,Electronics Store,Gym / Fitness Center,Gym,BBQ Joint
6,Gascoigne,Supermarket,BBQ Joint,Bus Station,Market,Chinese Restaurant,Electronics Store,History Museum,Gym / Fitness Center,Gym,Grocery Store
11,Parsloes,Pharmacy,Pub,Plaza,Construction & Landscaping,Gym,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop,Fast Food Restaurant
14,Valence,Bar,Pub,Business Service,Chinese Restaurant,History Museum,Park,Outdoors & Recreation,Train Station,Dry Cleaner,Gym


#### Cluster 3

In [403]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbey,Grocery Store,Platform,Fast Food Restaurant,Supermarket,Coffee Shop,Chinese Restaurant,Gym,Pharmacy,Pizza Place,Hotel
1,Alibon,Grocery Store,Bus Stop,Pub,Pizza Place,Convenience Store,Train Station,Cosmetics Shop,Gym / Fitness Center,Gym,Golf Course
2,Becontree,Dry Cleaner,Bus Stop,Fish & Chips Shop,Supermarket,Cosmetics Shop,Train Station,History Museum,Gym / Fitness Center,Gym,Grocery Store
3,Chadwell Heath,Hotel,Indian Restaurant,Grocery Store,Restaurant,Mini Golf,Cosmetics Shop,Gym,Golf Course,Gas Station,Fish & Chips Shop
7,Goresbrook,Soccer Field,Gym / Fitness Center,Gym,Movie Theater,Pizza Place,Convenience Store,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop
8,Heath,Supermarket,Gym / Fitness Center,Grocery Store,Bus Station,Golf Course,Martial Arts Dojo,Gas Station,Pool,Cosmetics Shop,Gym
9,Longbridge,Pharmacy,Grocery Store,Indian Restaurant,Pub,Pizza Place,Convenience Store,Gym,Golf Course,Gas Station,Fish & Chips Shop
13,Thames,Home Service,IT Services,Grocery Store,Nature Preserve,Train Station,Gym / Fitness Center,Gym,Golf Course,Gas Station,Fish & Chips Shop
16,Whalebone,Pharmacy,Pizza Place,Grocery Store,Fast Food Restaurant,Electronics Store,Supermarket,Bus Stop,Business Service,Gym / Fitness Center,Gym


#### Cluster 4

In [404]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Eastbrook,Lake,Park,Sporting Goods Shop,Construction & Landscaping,Train Station,Gym,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop
10,Mayesbrook,Park,Café,Construction & Landscaping,Train Station,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop


#### Cluster 5

In [405]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,River,Train Station,Platform,Convenience Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop,Fast Food Restaurant


In [434]:
london_merged_new = london_merged[['Neighborhood','Cluster Labels']]
london_merged_new.rename(columns={'Neighborhood':'Ward'},
                         inplace=True)
london_merged_new.drop(['District','Ward Latitude', 'Ward Longitude'],axis=1, inplace=True)
london_merged_new.head(1)

,Ward,Cluster Labels
0,Abbey,2


Put the clustered dataframe and the old one together and check it

In [435]:
df_final = pd.merge(london_merged_new, 
                    df_full, 
                    how='left', 
                    on='Ward')
df_final

,Ward,Cluster Labels,District Code,Ward Code,District,Rural/urban,District Latitude,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
0,Abbey,2,E09000002,E05000026,Barking and Dagenham,Urban major conurbation,51.546501,0.125055,51.539822,0.081291,209000,78188,57,32.9,17019.0,292910,180
1,Abbey,2,E09000024,E05000455,Merton,Urban major conurbation,51.411098,-0.196227,51.413528,-0.191540,208100,84201,55,36.7,12620.0,516974,205
2,Alibon,2,E09000002,E05000027,Barking and Dagenham,Urban major conurbation,51.546501,0.125055,51.545921,0.150987,209000,78188,57,32.9,17019.0,292910,180
3,Becontree,2,E09000002,E05000028,Barking and Dagenham,Urban major conurbation,51.546501,0.125055,51.552601,0.116912,209000,78188,57,32.9,17019.0,292910,180
4,Chadwell Heath,2,E09000002,E05000029,Barking and Dagenham,Urban major conurbation,51.546501,0.125055,51.583420,0.138596,209000,78188,57,32.9,17019.0,292910,180
5,Eastbrook,3,E09000002,E05000030,Barking and Dagenham,Urban major conurbation,51.546501,0.125055,51.555191,0.173453,209000,78188,57,32.9,17019.0,292910,180
6,Eastbury,1,E09000002,E05000031,Barking and Dagenham,Urban major conurbation,51.546501,0.125055,51.535809,0.104756,209000,78188,57,32.9,17019.0,292910,180
7,Gascoigne,1,E09000002,E05000032,Barking and Dagenham,Urban major conurbation,51.546501,0.125055,51.532581,0.085704,209000,78188,57,32.9,17019.0,292910,180
8,Goresbrook,2,E09000002,E05000033,Barking and Dagenham,Urban major conurbation,51.546501,0.125055,51.536041,0.132641,209000,78188,57,32.9,17019.0,292910,180
9,Heath,2,E09000002,E05000034,Barking and Dagenham,Urban major conurbation,51.546501,0.125055,51.558689,0.152931,209000,78188,57,32.9,17019.0,292910,180


#### Cluster 1

In [425]:
df_final.loc[df_final['Cluster Labels'] == 0, 
             df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Cluster Labels,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,District Code,Ward Code,...,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
16,0,Train Station,Dry Cleaner,History Museum,Gym / Fitness Center,Gym,Grocery Store,Golf Course,E09000002,E05000041,...,0.125055,51.536591,0.166611,209000,78188,57,32.9,17019.0,292910,180
17,0,Train Station,Dry Cleaner,History Museum,Gym / Fitness Center,Gym,Grocery Store,Golf Course,E09000024,E05000472,...,-0.196227,51.425541,-0.231880,208100,84201,55,36.7,12620.0,516974,205


#### Cluster 2

In [419]:
df_final.loc[df_final['Cluster Labels'] == 1, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Cluster Labels,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,District Code,Ward Code,...,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
6,1,Metro Station,History Museum,Bus Station,Electronics Store,Gym / Fitness Center,Gym,BBQ Joint,E09000002,E05000031,...,0.125055,51.535809,0.104756,209000,78188,57,32.9,17019.0,292910,180
7,1,Market,Chinese Restaurant,Electronics Store,History Museum,Gym / Fitness Center,Gym,Grocery Store,E09000002,E05000032,...,0.125055,51.532581,0.085704,209000,78188,57,32.9,17019.0,292910,180
12,1,Construction & Landscaping,Gym,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop,Fast Food Restaurant,E09000002,E05000037,...,0.125055,51.549831,0.135348,209000,78188,57,32.9,17019.0,292910,180
15,1,Chinese Restaurant,History Museum,Park,Outdoors & Recreation,Train Station,Dry Cleaner,Gym,E09000002,E05000040,...,0.125055,51.561230,0.135925,209000,78188,57,32.9,17019.0,292910,180


#### Cluster 3

In [420]:
df_final.loc[df_final['Cluster Labels'] == 2, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Cluster Labels,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,District Code,Ward Code,...,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
0,2,Supermarket,Coffee Shop,Chinese Restaurant,Gym,Pharmacy,Pizza Place,Hotel,E09000002,E05000026,...,0.125055,51.539822,0.081291,209000,78188,57,32.9,17019.0,292910,180
1,2,Supermarket,Coffee Shop,Chinese Restaurant,Gym,Pharmacy,Pizza Place,Hotel,E09000024,E05000455,...,-0.196227,51.413528,-0.191540,208100,84201,55,36.7,12620.0,516974,205
2,2,Pizza Place,Convenience Store,Train Station,Cosmetics Shop,Gym / Fitness Center,Gym,Golf Course,E09000002,E05000027,...,0.125055,51.545921,0.150987,209000,78188,57,32.9,17019.0,292910,180
3,2,Supermarket,Cosmetics Shop,Train Station,History Museum,Gym / Fitness Center,Gym,Grocery Store,E09000002,E05000028,...,0.125055,51.552601,0.116912,209000,78188,57,32.9,17019.0,292910,180
4,2,Restaurant,Mini Golf,Cosmetics Shop,Gym,Golf Course,Gas Station,Fish & Chips Shop,E09000002,E05000029,...,0.125055,51.583420,0.138596,209000,78188,57,32.9,17019.0,292910,180
8,2,Movie Theater,Pizza Place,Convenience Store,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop,E09000002,E05000033,...,0.125055,51.536041,0.132641,209000,78188,57,32.9,17019.0,292910,180
9,2,Bus Station,Golf Course,Martial Arts Dojo,Gas Station,Pool,Cosmetics Shop,Gym,E09000002,E05000034,...,0.125055,51.558689,0.152931,209000,78188,57,32.9,17019.0,292910,180
10,2,Pub,Pizza Place,Convenience Store,Gym,Golf Course,Gas Station,Fish & Chips Shop,E09000002,E05000035,...,0.125055,51.544430,0.093644,209000,78188,57,32.9,17019.0,292910,180
14,2,Nature Preserve,Train Station,Gym / Fitness Center,Gym,Golf Course,Gas Station,Fish & Chips Shop,E09000002,E05000039,...,0.125055,51.524078,0.119742,209000,78188,57,32.9,17019.0,292910,180
18,2,Fast Food Restaurant,Electronics Store,Supermarket,Bus Stop,Business Service,Gym / Fitness Center,Gym,E09000002,E05000042,...,0.125055,51.568272,0.141911,209000,78188,57,32.9,17019.0,292910,180


#### Cluster 4

In [421]:
df_final.loc[df_final['Cluster Labels'] == 3, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Cluster Labels,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,District Code,Ward Code,...,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
5,3,Construction & Landscaping,Train Station,Gym,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop,E09000002,E05000030,...,0.125055,51.555191,0.173453,209000,78188,57,32.9,17019.0,292910,180
11,3,Train Station,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop,E09000002,E05000036,...,0.125055,51.545189,0.120332,209000,78188,57,32.9,17019.0,292910,180


#### Cluster 5

In [422]:
df_final.loc[df_final['Cluster Labels'] == 4, df_final.columns[[1] + list(range(5, df_final.shape[1]))]]

,Cluster Labels,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,District Code,Ward Code,...,District Longitude,Ward Latitude,Ward Longitude,GLA Population Estimate 2017,GLA Household Estimate 2017,Population density (per hectare) 2017,"Average Age, 2017",Crime volume in 2017,AVG price on 2017-12-01,Sales volume on 2017-12-01
13,4,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gas Station,Fish & Chips Shop,Fast Food Restaurant,E09000002,E05000038,...,0.125055,51.524502,0.152645,209000,78188,57,32.9,17019.0,292910,180


Here we go.

Since I used filtered data most of the features was fine with me.
Add the information of Venue-s I think **Cluster 2** could work for me.

There are bus and metro station around, Gym, Park, Museum, Store...

So I should start here the searching.